In [80]:
import uuid
import random

from tqdm import tqdm
from threading import Thread

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *

In [81]:
CLUSTER_IDS = ['172.19.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS)
session = cluster.connect()
session.set_keyspace(KEYSPACE)

cluster.default_retry_policy = RetryPolicy()
cluster.default_reconnection_policy = ExponentialReconnectionPolicy(base_delay=1, max_delay=60, max_attempts=60)

In [82]:
res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    reservation_id = r.reservation_id
    
reservation_id

9e596e55-5cdd-41bd-9492-cf31fcf45809 4ec9d5f6-3af1-4503-93a6-7ff2ad6175e4 User 101 Book 2001


UUID('9e596e55-5cdd-41bd-9492-cf31fcf45809')

In [18]:
res = get_all_books(session)
print(len(list(res)))
for r in res:
    if r.is_reserved:
        print(r.book_id, r.book_name, r.is_reserved)
    if r.book_name == "Book 1":
        book_id = r.book_id
        book_name = "Book 1"

book_id, book_name

1001


(UUID('78f7e1da-9552-4e91-88d7-bc00021751c7'), 'Book 1')

In [10]:
res = get_all_users(session)
for r in res:
    if r.user_name == "User 1":
        user_name = "User 1"
        user_id = r.user_id
    if r.reservation_ids_list is not None:
        print(r.user_id, r.user_name, r.reservation_ids_list)

user_name, user_id

9a72123f-0ecf-4e24-ac69-530fe1c44467 User 101 [UUID('44c4f1be-d266-437b-9dcf-bbfe45eec299')]


('User 1', UUID('a26c83c2-3a1b-4f26-b18c-a0a8385bc492'))

In [83]:
def test_reserve_cancel_debug(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    # for _ in range(repeat_num):
    for _ in range(3):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)

        print("Created")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)

        cancel_reservation(session, reservation_id)
        print("Cancelled")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)


def test_update_reservations_debug(session, num_reservations=1000):
    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        assert reservation.book_id != new_book.book_id
        assert reservation.book_name != new_book.book_name

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)
        
        updated_reservation = get_reservation_by_id(session, reservation.reservation_id)
        
        assert updated_reservation.book_id == new_book.book_id
        assert updated_reservation.book_name == new_book.book_name

In [84]:
def test_same_request(session, reservation_id, new_book_id, repeat_num=10_000):
    # Runs an update of the book in a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        update_reservation_book(session, reservation_id, new_book_id)


def test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)


def test_update_reservations(session, num_reservations=1000):
    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)


def test_make_all_reservations_two_users(session, user1, user2):
    def make_all_reservations(user_id, user_name):
        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        books = get_all_books(session)
        available_books = [book for book in books if not book.is_reserved]
    
        while len(available_books) > 0:
            selected_random_book = random.choice(available_books)
            try:
                add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
            
            except:
                pass
            
            books = get_all_books(session)
            available_books = [book for book in books if not book.is_reserved]

        user_session.shutdown()

    thread1 = Thread(target=make_all_reservations, args=[user1.user_id, user1.user_name])
    thread2 = Thread(target=make_all_reservations, args=[user2.user_id, user2.user_name])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [89]:
users = get_all_users(session)
user1, user2 = None, None
for user in users:
    if user.user_name == "User 1":
        user1 = user

    if user.user_name == "User 2":
        user2 = user

test_make_all_reservations_two_users(session, user1, user2)

In [90]:
all_reservations = get_all_reservations(session)

user_1_reservations = 0
user_2_reservations = 0
for reservation in all_reservations:
    if reservation.user_id == user1.user_id:
        user_1_reservations += 1

    if reservation.user_id == user2.user_id:
        user_2_reservations += 1

print(f"User 1 reservation count: {user_1_reservations}")
print(f"User 2 reservation count: {user_2_reservations}")

User 1 reservation count: 1002
User 2 reservation count: 1002


In [91]:
res = list(get_all_reservations(session))
user_1_reservations = [r for r in res if r.user_id == user1.user_id]
user_2_reservations = [r for r in res if r.user_id == user2.user_id]

user_1 = get_user(session, user1.user_id)
user_2 = get_user(session, user2.user_id)

for r in user_1_reservations:
    if r in user_2_reservations:
        print("Error user1 reservation in user2 reservations list")

    if r.reservation_id not in user_1.reservation_ids_list:
        print("Error user1 reservation not in list")

for r in user_2_reservations:
    if r in user_1_reservations:
        print("Error user2 reservation in user1 reservations list")

    if r.reservation_id not in user_2.reservation_ids_list:
        print("Error user2 reservation not in list")

In [92]:
book_ids_1 = set([r.book_id for r in user_1_reservations])
book_ids_2 = set([r.book_id for r in user_2_reservations])

assert len(book_ids_1 & book_ids_2) == 0, "No book should be reserved by both clients at the same time"

AssertionError: No book should be reserved by both clients at the same time

In [93]:
book_ids_1 & book_ids_2

{UUID('35f42768-c776-44b8-8792-bee48d439181'),
 UUID('90c86772-d5b2-4638-82c5-7203c2c58674'),
 UUID('a5158b5f-3b9d-43a2-b059-49aea66b7886'),
 UUID('baa8dda6-ce01-47be-a147-8b3cc55bc330')}

In [96]:
book = list(book_ids_1 & book_ids_2)[0]
for res in get_all_reservations(session):
    if res.book_id == book:
        print(res.reservation_id, res.user_name, res.book_name, res.book_id)

eda0d700-677c-4df0-b291-6d3a82598039 User 1 Book 310 35f42768-c776-44b8-8792-bee48d439181
4fe17768-3b14-4e7b-8a66-142f6eb91b06 User 2 Book 310 35f42768-c776-44b8-8792-bee48d439181


In [94]:
books = get_all_books(session)
available_books = [book for book in books if not book.is_reserved]
available_books

[]

In [32]:
test_update_reservations(session)

100%|██████████| 1000/1000 [00:12<00:00, 77.19it/s]


In [15]:
test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    

100%|██████████| 10000/10000 [03:33<00:00, 46.90it/s]


44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1001


In [17]:
test_same_request(session, reservation_id, book_id)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)

100%|██████████| 10000/10000 [00:44<00:00, 226.46it/s]

44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1


In [ ]:
session.shutdown()
cluster.shutdown()